<a href="https://colab.research.google.com/github/pharaburda/MasterThesis/blob/master/SynergicDeepModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dask

In [ ]:
!pip install graphviz

In [ ]:
!pip install toolz

In [ ]:
import torch
import torchvision.models as models
from torch import nn
import math

#https://pytorch.org/tutorials/intermediate/model_parallel_tutorial.html

class Synergic(nn.Module):
    __constants__ = ['in_features', 'out_features']

    def __init__(self, in_features, out_features, bias=True):
        super(Synergic, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        return nn.functional.linear(input.to('cuda:0'), self.weight, self.bias)

    def extra_repr(self):
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )

def initialize_model(num_classes, use_pretrained=True):
    torch.manual_seed(10)
    first_component = models.resnet50(pretrained=use_pretrained).to('cuda:0')
    second_component = models.resnet50(pretrained=use_pretrained).to('cpu')

    num_ftrs = first_component.fc.in_features
    synergic_layer = Synergic(num_ftrs, num_classes)

    first_component.fc = synergic_layer.to('cuda:0')
    second_component.fc = synergic_layer.to('cuda:0')

    return first_component, second_component, synergic_layer

first_component, second_component, synergic_layer = initialize_model(5)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [ ]:
 torch.cuda.device_count()

1

In [ ]:
from torchvision import datasets
from torchvision.transforms import transforms
import os
input_size = 224

data_dir = '/content/drive/My Drive/Glaucoma'

transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

image_dataset_1 = datasets.ImageFolder(os.path.join(data_dir, 'train1'), transform)

train_dataset_1 = torch.utils.data.DataLoader(
        image_dataset_1, shuffle=False, num_workers=16
    )

for data, target in train_dataset_1:
  out = first_component(data.to('cuda:0'))


image_dataset_2 = datasets.ImageFolder(os.path.join(data_dir, 'train2'), transform)

image_dataset_2 = torch.utils.data.DataLoader(
        image_dataset_1, shuffle=False, num_workers=16
    )

for data, target in image_dataset_2:
  out = second_component(data.to('cpu'))

In [1]:
import os
glaucoma_dir = '/content/drive/My Drive/magisterka/glaucoma/train'
glaucoma_len= len([name for name in os.listdir(glaucoma_dir) if os.path.isfile(os.path.join(glaucoma_dir, name))])
print(glaucoma_len) 

retinopathy_dir = '/content/drive/My Drive/magisterka/diabetic retinopathy/train'
retinopathy_len= len([name for name in os.listdir(retinopathy_dir) if os.path.isfile(os.path.join(retinopathy_dir, name))])
print(retinopathy_len) 

amd_dir = '/content/drive/My Drive/magisterka/amd/train'
amd_len= len([name for name in os.listdir(amd_dir) if os.path.isfile(os.path.join(amd_dir, name))])
print(amd_len) 

134
476
143


In [ ]:
from graphviz import Digraph
import torch
from torch.autograd import Variable

def make_dot(var, params=None):
    """ Produces Graphviz representation of PyTorch autograd graph
    Blue nodes are the Variables that require grad, orange are Tensors
    saved for backward in torch.autograd.Function
    Args:
        var: output Variable
        params: dict of (name, Variable) to add names to node that
            require grad (TODO: make optional)
    """
    if params is not None:
        assert isinstance(params.values()[0], Variable)
        param_map = {id(v): k for k, v in params.items()}

    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def size_to_str(size):
        return '('+(', ').join(['%d' % v for v in size])+')'

    def add_nodes(var):
        if var not in seen:
            if torch.is_tensor(var):
                dot.node(str(id(var)), size_to_str(var.size()), fillcolor='orange')
            elif hasattr(var, 'variable'):
                u = var.variable
                name = param_map[id(u)] if params is not None else ''
                node_name = '%s\n %s' % (name, size_to_str(u.size()))
                dot.node(str(id(var)), node_name, fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'next_functions'):
                for u in var.next_functions:
                    if u[0] is not None:
                        dot.edge(str(id(u[0])), str(id(var)))
                        add_nodes(u[0])
            if hasattr(var, 'saved_tensors'):
                for t in var.saved_tensors:
                    dot.edge(str(id(t)), str(id(var)))
                    add_nodes(t)
    add_nodes(var.grad_fn)
    return dot

import torch
from torch.autograd import Variable

x = Variable(torch.randn(1,3,224,2048))
y = second_component(x)
g = make_dot(y)
g.view()

'Digraph.gv.pdf'